# Mesogens with NP | Pressure Ramps

## Pressure: 1.1 -> 1.3

### Date: 12/11/19 | System P = 3.3, Expected value of $T_c$ : 9.91

In [6]:
#-----Importa los paquetes esenciales para correr la simulación.

from __future__ import division
import hoomd
import hoomd.md

In [7]:
#-----Define important variables

p_prev = 1.1;
p_new = 1.3;
temp = 9.0;
steps_ramp = 2e4;
run_time = steps_ramp;

In [8]:
#-----Declare the file from wich we'll extract the configuration of the system.

ramp_file = "T_CM&NP_" + str(temp) + "_P_" + str(p_prev) + "_ramp.gsd"

In [9]:
#-----Start from a gsd file

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Select the trayectory of the center of mass of particles

system = hoomd.init.read_gsd(ramp_file,frame = -1)

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.tree()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define some groups and make their union.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#-----Integrate using NPT

pressure = hoomd.variant.linear_interp(points = [(0,p_prev), (steps_ramp, p_new)])
npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = temp, tau = 2.0, tauP = 2.0, P = pressure);
npt.randomize_velocities(seed = 42)

#-----Define los archivos en los que se guardará la información de la simulación. 

log_file = "T_" + str(temp) + "_P_" + str(p_new) + "_ramp.log"
gsd_file = "T_" + str(temp) + "_P_" + str(p_new) + "_ramp.gsd" 
meso_gsd_file = "T_CM&NP_" + str(temp) + "_P_" + str(p_new) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = groupNP_mes,
               overwrite = True);

notice(2): Group "all" created containing 1010 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): Group "Nano_Particles" created containing 10 particles
notice(2): Group "rigid_center" created containing 1000 particles
notice(2): Group "NP_Mes" created containing 1010 particles


In [10]:
#-----Run the simulation.

hoomd.run(run_time)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9010
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 121285 / 141000 | TPS 28.3624 | ETA 00:11:35
Time 00:00:20 | Step 121575 / 141000 | TPS 28.9472 | ETA 00:11:11
Time 00:00:30 | Step 121831 / 141000 | TPS 25.5328 | ETA 00:12:30
Time 00:00:40 | Step 122113 / 141000 | TPS 28.0958 | ETA 00:11:12
Time 00:00:50 | Step 122377 / 141000 | TPS 26.3007 | ETA 00:11:48
Time 00:01:00 | Step 122647 / 141000 | TPS 26.9595 | ETA 00:11:20
Time 00:01:10 | Step 122933 / 141000 | TPS 28.5089 | ETA 00:10:33
Time 00:01:20 | Step 123195 / 141000 | TPS 26.1676 | ETA 00:11:20
Time 00:01:30 | Step 123453 / 141000 | TPS 25.6903 | ETA 00:11:23
Time 00:01:40 | Step 123731 / 141000 | TPS 27.7461 | ETA 00:10:22
Time 00:01:50 | Step 123999 / 141000 | TPS 26.7889 | ETA 00:10:34
Time 00:02:00 | Step 124286 / 1

In [11]:
#-----Get density.

meta = system.get_metadata()

In [12]:
#-----Get volume.

volume = system.box.get_volume()